In [1]:
#### run LASSO with WH and l1 reg. regression for already generated test train splits 

In [2]:
import pandas as pd
import numpy as np
import random
import time
from __future__ import print_function
from builtins import input

from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()


import warnings
warnings.filterwarnings("ignore")




table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/A5.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/iHiV4.pkl")
iHiV = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/iHiV5.pkl")
VHred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/VH4.pkl")
tableg = table.T

iFT = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/iFT4.pkl")

VH = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/VH5.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/B5.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(5):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(5):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']

#### get all data -- design matrix, steady state abundances, and list of wells present

sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v5/Gould-etal-5species-relative-abundance-Oct22.pkl")
#sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v7/Gould-etal-5species-relative-abundance-Feb23median.pkl")
sstv3 = sst.copy()

laml = [1e-100,1e-30,5e-30,1e-25,1e-20,5e-20,500e-20,1e-11,1e-10,1e-5,.01,.05,.06,.0650,.0675,.07,0.08,0.1,0.13,0.15,0.18,0.2,0.5,1.0,2.0]
betasact = []
for species in range(5):
    abunvec = pd.DataFrame(sstv3[species][species_ones[species]].values)
    betasact.append([species,np.dot(VHred,abunvec).ravel()])
betac = pd.DataFrame(betasact)

from sklearn.metrics import r2_score

g = np.array([[1, 0], [-1, 1]])

for i in range(3):
    g = np.block([[g, np.zeros(g.shape)], [-g, g]])

Gred = g
Xred = np.linalg.inv(Gred)

In [3]:
sst.T[:5].sum().sum()

30.998149500356988

In [4]:
sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']
bugnames = ["Lactobacillus plantarum","L. brevis", "Acetobacter pasteurianus", "A. tropicalis","A. orientalis,"]

opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 1000,
                         'RelStopTol': 1e-10, 'AutoRho': {'RsdlTarget': 1.0}})

n = 2**5
random_state = None
cv_splitsl = [3]

In [5]:
dataTR = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v6/traindataRELwill.pkl")
dataTE = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v6/testdataRELwill.pkl")

In [6]:
err = []
betaspred  = [] 
errlin  = [] 
for cv_splits in cv_splitsl: 
    datatrain = dataTR[dataTR["k-fold"]==cv_splits]
    datatest = dataTE[dataTE["k-fold"]==cv_splits]
    for species in range(5):
        datatousetrain = datatrain[datatrain["species"]==species]
        datatousetest = datatest[datatest["species"]==species]
        abunvec = pd.DataFrame(sstv3[species][species_ones[species]].values)
        for lmda in laml:
            for iterii in range(cv_splits):
                rinew = datatousetrain[datatousetrain["fold number"]==iterii]["train data"].values[0]
                rileft = datatousetest[datatousetest["fold number"]==iterii]["test data"].values[0]
                y3 = abunvec.T[rinew].T[0].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = iHiVred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(iHiVred,x.ravel())
                err.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits])
                betaspred.append([lmda,cv_splits,species,x.ravel()])
                y3 = abunvec.T[rinew].T[0].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = Xred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(Xred,x.ravel())
                errlin.append([lmda,species,yrecon[rileft],rileft,rinew,cv_splits])

In [7]:
errdf= pd.DataFrame(err)
betadf = pd.DataFrame(betaspred)
errlindf = pd.DataFrame(errlin)
errdf.to_pickle("rel_CS_WH.pkl")
errlindf.to_pickle("rel_l1_sigma.pkl")